In [79]:
import pandas as pd

# ROBOTA
nodes_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\Nodes_processed.csv"
roads_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\Roads_processed.csv"
bridges_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\df_bridges_2024_all.csv"
survey_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\surveys\Survey.csv"

# Load the CSV data into a DataFrame
df_nodes = pd.read_csv(nodes_path, encoding='utf-8', delimiter=',')
df_roads = pd.read_csv(roads_path, encoding='utf-8', delimiter=',')
df_bridges_all = pd.read_csv(bridges_path, encoding='utf-8', delimiter=',')
df_survey = pd.read_csv(survey_path, encoding='utf-8', delimiter=',')

C:\Users\relia\AppData\Local\Temp\ipykernel_20964\4111068820.py:10: DtypeWarning:

Columns (5,16) have mixed types. Specify dtype option on import or set low_memory=False.

C:\Users\relia\AppData\Local\Temp\ipykernel_20964\4111068820.py:11: DtypeWarning:

Columns (10,11,12,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.



In [80]:
# Assuming df_bridges_all is your DataFrame
columns_to_convert = ['Zemepisná_dĺžka', 'Zemepisná_šírka', 'Výška', 'Normálna_zaťažiteľnosť',
                      'Výhradná_zaťažiteľnosť_t', 'Výnimočná_zaťažiteľnosť_t', 'Počet_otvorov',
                      'Dĺžka_nosnej_konštrukcie_m', 'Dĺžka_premostenia', 'Volná_šírka_mosta',
                      'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2', 'Plocha_mosta_m2']

# Convert specified columns to float
df_bridges_all[columns_to_convert] = df_bridges_all[columns_to_convert].apply(
    pd.to_numeric, errors='coerce')

# Convert Rok_postavenia to integer, assign NaN if not possible
df_bridges_all['Rok_postavenia'] = pd.to_numeric(
    df_bridges_all['Rok_postavenia'], errors='coerce')

df_bridges_all = df_bridges_all[df_bridges_all["n_2024"] != 0]

df_bridges_all.columns

Index(['ID_mosta', 'Trieda_PK', 'Číslo_PK', 'Názov_mosta', 'správcovské_číslo',
       'Zemepisná_šírka', 'Zemepisná_dĺžka', 'Výška', 'Druh_konštrukcie',
       'Material', 'Normálna_zaťažiteľnosť', 'Výhradná_zaťažiteľnosť_t',
       'Výnimočná_zaťažiteľnosť_t', 'Spôsob_stanovenia', 'Predmet_premostenia',
       'Rok_postavenia', 'Zaťažovacia_trieda_STN', 'Počet_otvorov',
       'Dĺžka_premostenia', 'Dĺžka_nosnej_konštrukcie_m', 'Volná_šírka_mosta',
       'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2',
       'Plocha_mosta_m2', 'Vlastník_úseku', 'Správca_úseku', 'Okres',
       'Počiatočný_uzol', 'Koncový_uzol', 'úsek_ID', 'Kraj', 'n_2012',
       'n_2013', 'n_2014', 'n_2015', 'n_2016', 'n_2017', 'n_2018', 'n_2019',
       'n_2020', 'n_2021', 'n_2022', 'n_2023', 'n_2024'],
      dtype='object')

In [81]:
df_bridges_all["Trieda_PK"].value_counts()

Trieda_PK
cesta III. triedy                      3967
cesta I. triedy                        1913
cesta II. triedy                       1472
diaľnica                                771
miestna neurčená                         56
účelová cesta (UC)                       13
II. trieda - miestna zberná (MZ)          2
III. trieda - miestna obslužná (MO)       1
Name: count, dtype: int64

In [82]:
filtered_bridges_I = df_bridges_all[
    (df_bridges_all["Trieda_PK"] == 'cesta I. triedy') 
]

filtered_bridges_II = df_bridges_all[
    (df_bridges_all["Trieda_PK"] == 'cesta II. triedy')
]

filtered_bridges_III = df_bridges_all[
    (df_bridges_all["Trieda_PK"] == 'cesta III. triedy')
]

filtered_bridges_I = filtered_bridges_I[filtered_bridges_I["n_2024"] != 0]
filtered_bridges_II = filtered_bridges_II[filtered_bridges_II["n_2024"] != 0]
filtered_bridges_III = filtered_bridges_III[filtered_bridges_III["n_2024"] != 0]

filtered_bridges_I.shape, filtered_bridges_II.shape, filtered_bridges_III.shape

((1913, 44), (1472, 44), (3967, 44))

In [83]:
df_bridges_all["n_2024"].value_counts()

n_2024
4    3035
5    1931
3    1381
6     895
2     525
1     368
7      60
Name: count, dtype: int64

In [84]:
import plotly.graph_objects as go
import plotly.io as pio

# Creating the histogram for each category with offset groups
trace_filtered_I = go.Histogram(
    x=filtered_bridges_I["n_2024"],
    name='mosty ciest I. triedy',
    marker_color='#e7a564',
    opacity=0.75,
    histnorm='percent',
    offsetgroup=1
)

trace_filtered_II = go.Histogram(
    x=filtered_bridges_II["n_2024"],
    name='mosty ciest II. triedy',
    marker_color='#46a7a2',
    opacity=0.75,
    histnorm='percent',
    offsetgroup=2
)

trace_filtered_III = go.Histogram(
    x=filtered_bridges_III["n_2024"],
    name='mosty ciest III. triedy',
    marker_color='#788BFF',
    opacity=0.75,
    histnorm='percent',
    offsetgroup=3
)

# Creating the figure and adding the histograms
fig = go.Figure(data=[trace_filtered_I, trace_filtered_II, trace_filtered_III])

# Update the layout with Montserrat font
fig.update_layout(
    title_text='Relatívne zastúpenie STS mostov',
    xaxis_title_text='STS',
    yaxis_title_text='zastúpenie (%)',
    barmode='group',  # Grouped bars
    font=dict(
        family="Calibri, sans-serif",
        size=14
    ),
    legend=dict(
        orientation="v",
        yanchor="top",
        y=1.0,
        xanchor="right",
        x=1
    ),
    plot_bgcolor='#EEEEEE',
    margin=dict(l=0, r=20, t=50, b=50),
    bargap=0.2
)

# Define manual x-offsets for each group
x_offsets = [-0.3, 0, 0.3]  # Offset for each group (I, II, III)

# Adding annotations for absolute values at the top of each bar
for i, (trace, data) in enumerate(zip(
    [trace_filtered_I, trace_filtered_II, trace_filtered_III],
    [filtered_bridges_I, filtered_bridges_II, filtered_bridges_III]
)):
    counts = data["n_2024"].value_counts().sort_index()
    for x, count in counts.items():
        fig.add_annotation(
            x=x + x_offsets[i],  # Use the specified offset for each group
            # Position at the top of the bar in percentage
            y=(count / data["n_2024"].count()) * 100,
            text=str(count),  # Absolute count as text
            showarrow=False,
            yshift=10,
            font=dict(size=12, color="black")
        )

# Set desired resolution (width and height in pixels)
width = 600  # e.g., 1200 pixels wide
height = 600  # e.g., 800 pixels high

# Update layout with the desired width and height
fig.update_layout(width=width, height=height)

# Show the plot
fig.show()


# Save the figure as SVG
pio.write_image(fig, "Kategorie.svg", format="svg",
                width=width, height=height)

In [85]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# Define a function to calculate relative condition counts


def calculate_relative_condition_counts(data, start_year=2012, end_year=2024):
    relative_counts = pd.DataFrame()

    for year in range(start_year, end_year + 1):
        column_name = f"n_{year}"
        year_data = data[data[column_name] > 0]  # Exclude bridges not assessed
        counts = year_data[column_name].value_counts().rename_axis(
            'Condition').reset_index(name=column_name)
        total = counts[column_name].sum()
        counts[column_name] = (counts[column_name] / total) * 100

        if relative_counts.empty:
            relative_counts = counts
        else:
            relative_counts = pd.merge(
                relative_counts, counts, on='Condition', how='outer')

    relative_counts.fillna(0, inplace=True)
    relative_counts.sort_values('Condition', inplace=True)
    return relative_counts


# Calculate relative condition counts for each category
relative_condition_counts_I = calculate_relative_condition_counts(
    filtered_bridges_I)
relative_condition_counts_II = calculate_relative_condition_counts(
    filtered_bridges_II)
relative_condition_counts_III = calculate_relative_condition_counts(
    filtered_bridges_III)

# Define the function to create the plot


def create_plot(relative_counts, title, filename):
    color_palette = ['#46a7a2', '#e7a564', '#8bae7c',
                     '#353a29', '#eae1a9', '#c25469', '#2d505d']
    data_for_plotting = relative_counts.set_index('Condition').T.reset_index()
    data_for_plotting.rename(columns={'index': 'Year'}, inplace=True)
    years = ['{}'.format(year) for year in range(2012, 2025)]
    data_for_plotting['Year'] = years

    fig = go.Figure()

    for i, condition in reversed(list(enumerate(relative_counts['Condition'].unique()))):
        fig.add_trace(go.Scatter(
            x=data_for_plotting['Year'],
            y=data_for_plotting[condition],
            mode='lines',
            stackgroup='one',
            name=str(condition),
            line=dict(color=color_palette[i % len(color_palette)])
        ))

    fig.update_layout(
        title=title,
        yaxis_title='percentuálne zastúpenie',
        yaxis=dict(ticksuffix="%", range=[0, 100]),
        font=dict(family="Calibri, sans-serif", size=14),
        legend=dict(
            orientation="h",
            yanchor="top",
            y=1.080,
            xanchor="right",
            x=1
        ),
        xaxis=dict(
            tickvals=[str(year) for year in range(2012, 2025)],
            ticktext=["'12", "'13", "'14", "'15", "'16", "'17",
                      "'18", "'19", "'20", "'21", "'22", "'23", "'24"]
        ),
        plot_bgcolor='#EEEEEE',
        margin=dict(l=0, r=20, t=75, b=25),
        width=500,
        height=500
    )

    fig.show()
    pio.write_image(fig, f"{filename}.svg",
                    format="svg", width=500, height=500)


# Create and save plots for each category
create_plot(relative_condition_counts_I,
            'Vývoj STS mostov na cestách I. triedy', 'Vyvoj_I')
create_plot(relative_condition_counts_II,
            'Vývoj STS mostov na cestách II. triedy', 'Vyvoj_II')
create_plot(relative_condition_counts_III,
            'Vývoj STS mostov na cestách III. triedy', 'Vyvoj_III')

In [86]:
import plotly.graph_objects as go

# Define the color palette
color_palette = ['#e7a564', '#46a7a2', '#c6d9df', '#b17431',
                 '#8bae7c', '#353a29', '#f1aab2', '#eae1a9',
                 '#c25469', '#2d505d'
                 ]

# Get the material counts
material_counts_I = filtered_bridges_I["Material"].value_counts()

# Create a bar plot
fig = go.Figure()

# Add traces for each material, include the count as text, and set textposition to 'auto' for automatic placement
for material, count, color in zip(material_counts_I.index, material_counts_I.values, color_palette):
    fig.add_trace(go.Bar(
        x=[material],  # Set x to material for vertical bars
        y=[count],
        name=material,
        marker_color=color,
        text=[count],  # Add the count as text for each bar
        textposition='outside'  # Position the text outside of bars
    ))

# Update the layout with additional configurations
fig.update_layout(
    title="Konštrukčné materiály mostov na cestách I. triedy",
    yaxis_title="počet",
    xaxis=dict(
        showticklabels=False
    ),
    plot_bgcolor='#EEEEEE',  # Set plot background color
    font=dict(family="Calibri, sans-serif",
              size=14),  # Set font family and size
    legend=dict(
        orientation="h",  # Horizontal legend orientation
        yanchor="top",  # Anchor legend to the top
        y=0.975,  # Position the legend slightly above the plot
        xanchor="right",  # Anchor legend to the right
        x=0.985  # Align legend to the right
    ),
    margin=dict(l=0, r=20, t=50, b=30),  # Adjust margins for tight layout
    bargap=0.3,  # Increase the gap between bars for more space
)

# Add a custom x-axis label using annotations
fig.update_layout(
    annotations=[
        dict(
            x=0.5,  # x-coordinate position (0.5 for center)
            y=-0.075,  # y-coordinate position (-0.15 places it below the bars)
            xref='paper',  # Use 'paper' for positioning relative to the plot area
            yref='paper',  # Use 'paper' for positioning relative to the plot area
            showarrow=False,  # Do not show an arrow
            text="materiál",  # The text you want as the x-axis label
            # Match the font size with the rest of the plot
            font=dict(family="Calibri, sans-serif",
                      size=15),
        )
    ]
)

# Show the updated plot
fig.show()

In [87]:
material_counts_I = filtered_bridges_I["Material"].value_counts()
material_counts_II = filtered_bridges_II["Material"].value_counts()
material_counts_III = filtered_bridges_III["Material"].value_counts()

In [88]:
import plotly.graph_objects as go
import pandas as pd
import plotly.io as pio

# Define the full color palette for materials
color_palette = {
    'monolitický železobetón': '#e7a564',
    'prefabrikovaný predpätý betón': '#46a7a2',
    'prefabrikovaný železobetón': '#c6d9df',
    'monolitický predpätý betón': '#b17431',
    'monolitický prostý betón': '#8bae7c',
    'kameň': '#353a29',
    'oceľ -  plnostenná': '#f1aab2',
    'spriahnutá betón-betón': '#eae1a9',
    'Nezistený': '#c25469',
    'iné materiály': '#2d505d',
    'oceľobetón': '#a2a2d2',
    'spriahnutá oceľ-betón': '#b3d9ff',
    'oceľ -  priehradová': '#ffb6c1',
    'tehla': '#ffd700'
}

# Function to create and save a pie chart for a bridge class


def create_pie_chart(filtered_bridges, class_name, filename):
    # Get material counts and aggregate
    material_counts = filtered_bridges["Material"].value_counts()
    # Separate the top 5 materials and aggregate the rest into "Iné materiály"
    top_5_materials = material_counts.head(4)
    other_material_count = material_counts.iloc[4:].sum()
    aggregated_counts = pd.concat(
        [top_5_materials, pd.Series({"iné materiály": other_material_count})])

    # Map colors based on the order of labels in aggregated_counts
    colors = [color_palette.get(material, '#353a29')
              for material in aggregated_counts.index]

    # Create the pie chart
    fig = go.Figure(go.Pie(
        labels=aggregated_counts.index,
        values=aggregated_counts.values,
        marker=dict(colors=colors),
        textinfo='percent',  # Show only the percentage inside the slices
        insidetextorientation='radial'  # Text orientation for better readability
    ))

    # Update layout with figure size, margins, and legend settings
    fig.update_layout(
        title=f"Konštrukčné materiály mostov na {class_name}",
        font=dict(family="Calibri, sans-serif", size=14),
        showlegend=True,
        # Keeps legend order as per aggregated_counts order
        legend=dict(traceorder="normal"),
        width=500,  # Set figure width
        height=500,  # Set figure height
        margin=dict(l=20, r=20, t=40, b=20)  # Tighten up margins
    )

    # Show the pie chart
    fig.show()

    # Save the figure as SVG
    pio.write_image(fig, f"{filename}.svg", format="svg")


# Generate pie charts for each bridge class and save them as SVGs
create_pie_chart(filtered_bridges_I, "cestách I. triedy",
                 "material_pie_I_class")
create_pie_chart(filtered_bridges_II, "cestách II. triedy",
                 "material_pie_II_class")
create_pie_chart(filtered_bridges_III, "cestách III. triedy",
                 "material_pie_III_class")

In [89]:
df_bridges_all["Trieda_PK"].value_counts()

Trieda_PK
cesta III. triedy                      3967
cesta I. triedy                        1913
cesta II. triedy                       1472
diaľnica                                771
miestna neurčená                         56
účelová cesta (UC)                       13
II. trieda - miestna zberná (MZ)          2
III. trieda - miestna obslužná (MO)       1
Name: count, dtype: int64

In [90]:
df_bridges_all["Material"].value_counts()

Material
monolitický železobetón          3264
prefabrikovaný železobetón       1917
prefabrikovaný predpätý betón    1658
monolitický predpätý betón        359
oceľ -  plnostenná                268
monolitický prostý betón          229
kameň                             193
Nezistený                         105
spriahnutá betón-betón             77
spriahnutá oceľ-betón              31
oceľobetón                         27
tehla                              26
iný materiál                       22
oceľ -  priehradová                18
Name: count, dtype: int64

In [91]:
# Filter bridges with the material "prefabrikovaný predpätý betón" and road classes "cesta III. triedy" and "cesta II. triedy"
filtered_prefabricated_prestressed_concrete = df_bridges_all[
    (df_bridges_all["Material"] == "prefabrikovaný predpätý betón") &
    (df_bridges_all["Trieda_PK"].isin(
        ["cesta III. triedy", "cesta II. triedy"]))
]

# Display the filtered DataFrame
filtered_prefabricated_prestressed_concrete

,ID_mosta,Trieda_PK,Číslo_PK,Názov_mosta,správcovské_číslo,Zemepisná_šírka,Zemepisná_dĺžka,Výška,Druh_konštrukcie,Material,...,n_2015,n_2016,n_2017,n_2018,n_2019,n_2020,n_2021,n_2022,n_2023,n_2024
1324,M5833,cesta III. triedy,3450,Most nad traťou ŽSR v meste Prešov,49,48.990600,21.250670,246.300000,dosková,prefabrikovaný predpätý betón,...,4,4,5,5,6,6,6,6,6,6
1509,M6252,cesta II. triedy,507,"75_001 Most cez potok Šárd, k.ú. Galanta",1,48.186735,17.712248,119.005749,dosková,prefabrikovaný predpätý betón,...,4,4,4,4,4,4,4,5,5,5
1742,M433,cesta II. triedy,426,426-021 Most cez potok Chvojnica pred mestom H...,21,48.824790,17.167195,166.001999,trámová,prefabrikovaný predpätý betón,...,4,4,4,4,4,4,4,4,6,7
1753,M6600,cesta II. triedy,487,487_038 Most cez rieku Kysuca v obci Makov,38,49.371169,18.482524,583.144000,dosková,prefabrikovaný predpätý betón,...,5,5,5,5,5,5,5,5,5,1
1764,M3132,cesta II. triedy,487,487_054 Most cez potok Dlhá Dolina v obci Turz...,54,49.396726,18.619917,478.857250,dosková,prefabrikovaný predpätý betón,...,4,4,4,4,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8118,M10149,cesta III. triedy,3216,Most cez Podhorský potok pred obcou Bijacovce,1,49.007010,20.777680,459.618134,trámová,prefabrikovaný predpätý betón,...,0,0,0,0,0,0,0,0,4,4
8143,M10179,cesta III. triedy,1267,Cestný nadjazd - žst. Veľké Kostoľany,NaN,48.505650,17.772420,NaN,iná,prefabrikovaný predpätý betón,...,0,0,0,0,0,0,0,0,0,6
8144,M10176,cesta III. triedy,1279,cestný nadjazd nad traťou ŽSR,NaN,48.372860,17.631530,NaN,iná,prefabrikovaný predpätý betón,...,0,0,0,0,0,0,0,0,0,4
8151,M9901,cesta III. triedy,2137,MO2137-003 Most ponad privádzač v k.ú. Priekop...,3,49.097130,18.951390,NaN,trámová,prefabrikovaný predpätý betón,...,0,0,0,0,0,0,0,0,0,1


In [92]:
# Initialize an empty dictionary to store the describe outputs for each region
region_descriptions = {}

# Loop through each unique region in the "Kraj" column
for region in filtered_prefabricated_prestressed_concrete["Kraj"].unique():
    # Filter the DataFrame by region
    region_data = filtered_prefabricated_prestressed_concrete[
        filtered_prefabricated_prestressed_concrete["Kraj"] == region]
    # Perform describe on the "n_2024" column for the current region and store it in the dictionary
    region_descriptions[region] = region_data["n_2024"].value_counts()

# Display the descriptions for each region
region_descriptions

{'Prešov': n_2024
 5    130
 4    117
 3     32
 6     25
 2      6
 1      6
 7      3
 Name: count, dtype: int64,
 'Trnava': n_2024
 5    41
 4    23
 6     7
 7     4
 1     3
 2     1
 3     1
 Name: count, dtype: int64,
 'Žilina': n_2024
 4    65
 3    31
 5    21
 2    11
 6     7
 1     6
 Name: count, dtype: int64,
 'Trenčín': n_2024
 4    36
 5    19
 3     9
 6     7
 1     3
 2     1
 Name: count, dtype: int64,
 'Nitra': n_2024
 4    34
 5    12
 3     8
 6     2
 2     2
 7     1
 Name: count, dtype: int64,
 'Bratislava': n_2024
 4    9
 5    6
 6    5
 1    1
 3    1
 Name: count, dtype: int64,
 'Banská Bystrica': n_2024
 4    84
 5    27
 6    18
 3    14
 7     2
 2     2
 Name: count, dtype: int64,
 'Košice': n_2024
 6    42
 4    39
 5    30
 3     6
 2     5
 1     4
 7     3
 Name: count, dtype: int64}

In [93]:
import geopandas as gpd

file_path = r'C:\Users\relia\Documents\GitHub\Bridges\GeoJSONs\regions_epsg_4326.geojson'  # robota

gdf = gpd.read_file(file_path)

In [94]:
# Initialize an empty dictionary to store the sum of "Dĺžka_premostenia" for each region
region_bridge_lengths = {}

# Loop through each unique region in the "Kraj" column
for region in filtered_prefabricated_prestressed_concrete["Kraj"].unique():
    # Filter the DataFrame by region
    region_data = filtered_prefabricated_prestressed_concrete[
        filtered_prefabricated_prestressed_concrete["Kraj"] == region]
    # Calculate the sum of "Dĺžka_premostenia" for the current region and store it in the dictionary
    region_bridge_lengths[region] = region_data["Dĺžka_premostenia"].sum()

# Display the sums for each region
region_bridge_lengths

{'Prešov': np.float64(7790.982),
 'Trnava': np.float64(2835.107),
 'Žilina': np.float64(5038.3279999999995),
 'Trenčín': np.float64(2313.0870000000004),
 'Nitra': np.float64(2345.02),
 'Bratislava': np.float64(1074.9600000000003),
 'Banská Bystrica': np.float64(3860.462),
 'Košice': np.float64(4302.215)}

In [95]:
filtered_prefabricated_prestressed_concrete["Kraj"].value_counts()

Kraj
Prešov             319
Banská Bystrica    147
Žilina             141
Košice             129
Trnava              80
Trenčín             75
Nitra               59
Bratislava          22
Name: count, dtype: int64

In [96]:
# Filter each region manually and assign to a separate DataFrame
bratislavsky_bridges = filtered_prefabricated_prestressed_concrete[
    filtered_prefabricated_prestressed_concrete["Kraj"] == "Bratislava"
]

trnavsky_bridges = filtered_prefabricated_prestressed_concrete[
    filtered_prefabricated_prestressed_concrete["Kraj"] == "Trnava"
]

nitriansky_bridges = filtered_prefabricated_prestressed_concrete[
    filtered_prefabricated_prestressed_concrete["Kraj"] == "Nitra"
]

zilinsky_bridges = filtered_prefabricated_prestressed_concrete[
    filtered_prefabricated_prestressed_concrete["Kraj"] == "Žilina"
]

trenciansky_bridges = filtered_prefabricated_prestressed_concrete[
    filtered_prefabricated_prestressed_concrete["Kraj"] == "Trenčín"
]

banskobystricky_bridges = filtered_prefabricated_prestressed_concrete[
    filtered_prefabricated_prestressed_concrete["Kraj"] == "Banská Bystrica"
]

presovsky_bridges = filtered_prefabricated_prestressed_concrete[
    filtered_prefabricated_prestressed_concrete["Kraj"] == "Prešov"
]

kosicky_bridges = filtered_prefabricated_prestressed_concrete[
    filtered_prefabricated_prestressed_concrete["Kraj"] == "Košice"
]

# Check the shapes of each DataFrame
print("Bratislavský:", bratislavsky_bridges.shape)
print("Trnavský:", trnavsky_bridges.shape)
print("Nitriansky:", nitriansky_bridges.shape)
print("Žilinský:", zilinsky_bridges.shape)
print("Trenčiansky:", trenciansky_bridges.shape)
print("Banskobystrický:", banskobystricky_bridges.shape)
print("Prešovský:", presovsky_bridges.shape)
print("Košický:", kosicky_bridges.shape)

Bratislavský: (22, 44)
Trnavský: (80, 44)
Nitriansky: (59, 44)
Žilinský: (141, 44)
Trenčiansky: (75, 44)
Banskobystrický: (147, 44)
Prešovský: (319, 44)
Košický: (129, 44)


In [106]:
import geopandas as gpd
import folium


def create_region_map(region_name, bridges_df, output_file="region_map.html", icon_size=(15, 15)):
    """
    Creates a map with a specified region shape and bridge locations marked.

    Parameters:
        region_name (str): Name of the region to plot (matching the 'NM4' column in GeoDataFrame).
        bridges_df (DataFrame): DataFrame containing bridge data with 'Zemepisná_šírka' and 'Zemepisná_dĺžka' columns.
        output_file (str): Path to save the output HTML map (default is 'region_map.html').
        icon_size (tuple): Size of the icons in pixels, e.g., (15, 15).
    """
    # Filter to get only the specified region from the already loaded GeoDataFrame
    region_shape = gdf[gdf["NM4"] == region_name]

    # Initialize the Folium map centered on the region, without a base map
    region_center = region_shape.geometry.centroid.iloc[0]
    m = folium.Map(location=[region_center.y,
                   region_center.x], zoom_start=10, tiles=None)

    # Add the region shape to the map
    folium.GeoJson(
        region_shape,
        style_function=lambda x: {
            'fillColor': '#1E90FF',  # Fixed light blue fill color
            'color': 'black',         # Black boundary line
            'weight': 1,
            'fillOpacity': 0.0        # Transparent fill
        },
        name=f"{region_name} Region"
    ).add_to(m)

    # Add black pins with "i" for each bridge in the specified region without popups
    for _, row in bridges_df.iterrows():
        folium.Marker(
            location=[row['Zemepisná_šírka'], row['Zemepisná_dĺžka']],
            icon=folium.Icon(color="black", icon="info-sign",
                             prefix="glyphicon"),  # 'info-sign' icon for "i"
            icon_size=icon_size  # Set custom size
        ).add_to(m)

    # Save the map as an HTML file
    m.save(output_file)
    print(f"Map saved to {output_file}")



In [109]:
# Dictionary to map region names to their respective DataFrames
regions_dfs = {
    "Bratislavský": bratislavsky_bridges,
    "Trnavský": trnavsky_bridges,
    "Nitriansky": nitriansky_bridges,
    "Žilinský": zilinsky_bridges,
    "Trenčiansky": trenciansky_bridges,
    "Banskobystrický": banskobystricky_bridges,
    "Prešovský": presovsky_bridges,
    "Košický": kosicky_bridges
}

# Loop through each region in the dictionary and create a map
for region_name, bridges_df in regions_dfs.items():
    # Define the output file path, using the region name in the file name
    output_file = f"{region_name}_bridges_map.html"

    # Create the map for each region
    create_region_map(
        region_name=region_name,
        bridges_df=bridges_df,
        output_file=output_file,
        icon_size=(5, 5)
    )

C:\Users\relia\AppData\Local\Temp\ipykernel_20964\3265855793.py:19: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


C:\Users\relia\AppData\Local\Temp\ipykernel_20964\3265855793.py:19: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


C:\Users\relia\AppData\Local\Temp\ipykernel_20964\3265855793.py:19: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


C:\Users\relia\AppData\Local\Temp\ipykernel_20964\3265855793.py:19: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS befor

Map saved to Bratislavský_bridges_map.html
Map saved to Trnavský_bridges_map.html
Map saved to Nitriansky_bridges_map.html
Map saved to Žilinský_bridges_map.html
Map saved to Trenčiansky_bridges_map.html
Map saved to Banskobystrický_bridges_map.html


C:\Users\relia\AppData\Local\Temp\ipykernel_20964\3265855793.py:19: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


C:\Users\relia\AppData\Local\Temp\ipykernel_20964\3265855793.py:19: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


C:\Users\relia\AppData\Local\Temp\ipykernel_20964\3265855793.py:19: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




Map saved to Prešovský_bridges_map.html
Map saved to Košický_bridges_map.html


C:\Users\relia\AppData\Local\Temp\ipykernel_20964\3265855793.py:19: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




In [110]:
import plotly.graph_objects as go
import pandas as pd


def create_bar_plot(bridges_df, region_name, bar_color="#1E90FF", output_file="region_n_2024_distribution.svg", width=600, height=400):
    """
    Creates a bar plot for the distribution of "n_2024" values for a given region.

    Parameters:
        bridges_df (DataFrame): DataFrame containing bridge data with "n_2024" column.
        region_name (str): Name of the region to include in the title.
        bar_color (str): Hex color code for the bars.
        output_file (str): Path to save the output SVG plot.
        width (int): Width of the plot in pixels.
        height (int): Height of the plot in pixels.
    """
    # Get value counts and reindex to include all values from 1 to 7, filling missing values with 0
    value_counts = bridges_df["n_2024"].value_counts()
    all_values = pd.Series(range(1, 8), name="n_2024")
    ordered_counts = all_values.map(value_counts).fillna(0).astype(int)

    # Create the bar plot
    fig = go.Figure(go.Bar(
        x=all_values,
        y=ordered_counts,
        marker_color=bar_color  # Set bar color
    ))

    # Update layout with title, axis labels, and figure size
    fig.update_layout(
        title=f"Prefabrikované predpäté mosty - {region_name}",
        xaxis_title="STS",
        yaxis_title="Počet",
        width=width,  # Adjustable width
        height=height,  # Adjustable height
        margin=dict(l=20, r=20, t=40, b=20)
    )

    # Save the plot as an SVG
    fig.write_image(output_file, format="svg", width=width, height=height)
    print(f"Plot saved to {output_file}")

    # Show the plot
    fig.show()


In [111]:
# Define regions and their colors
regions_info = {
    "Bratislavský": {"data": bratislavsky_bridges, "color": "#1E90FF", "output": "Bratislava_bridges_n_2024_distribution.svg"},
    "Trnavský": {"data": trnavsky_bridges, "color": "#FFD700", "output": "Trnava_bridges_n_2024_distribution.svg"},
    "Nitriansky": {"data": nitriansky_bridges, "color": "#d45500", "output": "Nitra_bridges_n_2024_distribution.svg"},
    "Žilinský": {"data": zilinsky_bridges, "color": "#228B22", "output": "Zilina_bridges_n_2024_distribution.svg"},
    "Trenčiansky": {"data": trenciansky_bridges, "color": "#FF4500", "output": "Trencin_bridges_n_2024_distribution.svg"},
    "Banskobystrický": {"data": banskobystricky_bridges, "color": "#aa32cd", "output": "Banskobystrica_bridges_n_2024_distribution.svg"},
    "Prešovský": {"data": presovsky_bridges, "color": "#6A5ACD", "output": "Presov_bridges_n_2024_distribution.svg"},
    "Košický": {"data": kosicky_bridges, "color": "#FF6347", "output": "Kosice_bridges_n_2024_distribution.svg"}
}

# Loop through each region and create a bar plot
for region_name, info in regions_info.items():
    create_bar_plot(
        bridges_df=info["data"],
        region_name=region_name,
        bar_color=info["color"],
        output_file=info["output"],
        width=600,
        height=400
    )

Plot saved to Bratislava_bridges_n_2024_distribution.svg


Plot saved to Trnava_bridges_n_2024_distribution.svg


Plot saved to Nitra_bridges_n_2024_distribution.svg


Plot saved to Zilina_bridges_n_2024_distribution.svg


Plot saved to Trencin_bridges_n_2024_distribution.svg


Plot saved to Banskobystrica_bridges_n_2024_distribution.svg


Plot saved to Presov_bridges_n_2024_distribution.svg


Plot saved to Kosice_bridges_n_2024_distribution.svg
